In [1]:
import SimpleITK as sitk
import numpy as np
from glob import glob
import math
import matplotlib.pyplot as plt
import pydicom

In [2]:
### This folder contains a stack of micro-ultrasound dicom images
dicom_path = '../UF_Data/micro-ultrasound/Clippped/UF010/Tumor/1_Small/'
### Get the file names of all dicom images in the folder dicom_path
dicom_image_names = glob(dicom_path + '*.dcm')

num_of_images = len(dicom_image_names)

list_of_dicom_images = [] ### a list stores dicom images read by sitk
angles = np.zeros(num_of_images) ### an array stores the rotation angle of each dicom image

for i in range(num_of_images):
    img = sitk.ReadImage(dicom_image_names[i])
    list_of_dicom_images.append(img)
    angles[i] = img.GetOrigin()[2] ### the third coordinate of the image origin is the rotation angle in degree

### get the min and max rotation angles of the sweep
angle_max = np.max(angles)
angle_min = np.min(angles)
print("minimum angle: ", angle_min)
print("maximum angle: ", angle_max)

minimum angle:  -68.21099616261
maximum angle:  87.3178479169242


In [3]:
### number of dicom image pixels in the superior-inferior direction
width_num_pixels = list_of_dicom_images[0].GetSize()[0]
### number of dicom image pixels in the oblique direction
height_num_pixels = list_of_dicom_images[0].GetSize()[1]

### get in-plane resolution of dicom images
spacing = list_of_dicom_images[0].GetSpacing()[0]
### compute width of each dicom image (mm)
width = spacing*width_num_pixels
### compute height of each dicom image (mm)
height = spacing*height_num_pixels

### radius of the probe (mm)
probe_radius = 12.5 

#### compute the physical size of the reconstructed 3D micro-ultrasound volume in each direction 
AP = height + probe_radius # distance from anterior to posterior
LR = 2*AP # distance from left to right
SI = width # distance from superior to interior

In [4]:
#### since the in-plane resolution of micro-ultrasound dicom images is very high (~0.04mm),
### We can use isotropic voxel spacing of ~6*0.04 = 0.24mm
factor = 6
### Set spacing in each direction
spacing_LR = spacing*factor
spacing_AP = spacing*factor
spacing_SI = spacing*factor*4
### compute number of pixels in each direction based on the physical size and image spacing
LR_num_pixels = int(LR/spacing_LR)
AP_num_pixels = int(AP/spacing_AP)
SI_num_pixels = int(SI/spacing_SI)

#### creat a 3D micro-ultrasound image volume
microUS_3D = sitk.Image(LR_num_pixels,AP_num_pixels,SI_num_pixels,sitk.sitkUInt8)
### SetImage Spacing
microUS_3D.SetSpacing((spacing_LR, spacing_AP, spacing_SI ))

In [5]:
def get_rotation_angle(x,y,h,r):
    ### x, y are image indicies, they are integers
    ### h is the height of the input dicom image, r is the radius of the probe, unit is mm
    
    distance_to_midline = x*spacing_LR - (h+r)
    distance_to_posterior = h + r - y*spacing_AP
    
    theta = math.degrees(math.atan(distance_to_midline/distance_to_posterior))
    return -theta

def get_distance_to_center_of_probe(x,y,h,r):
    distance_to_midline = x*spacing_LR - (h+r)
    distance_to_posterior = h + r - y*spacing_AP
    
    return np.sqrt(distance_to_midline**2 + distance_to_posterior**2)

In [6]:
for z in range(SI_num_pixels):
    for x in range(LR_num_pixels):
        for y in range(AP_num_pixels):
            theta = get_rotation_angle(x,y,height,probe_radius)

            if theta >= angle_min and theta <= angle_max:
                ### find the micro-US image for theta
                idx = np.abs(angles - theta).argmin()

                ### get the i, and j index of the point (x,y,z) in the 2D microUS slice (idx)
                i = int(z*spacing_SI/spacing)
                d = get_distance_to_center_of_probe(x,y,height,probe_radius)
                j = int((height + probe_radius - d)/spacing)
                if (j>=0 and j<height_num_pixels) and i<width_num_pixels:
                    pixelValue = list_of_dicom_images[idx].GetPixel(i,j,0)[0]
                    microUS_3D.SetPixel( (x,y,z), pixelValue)  

In [7]:
sitk.WriteImage(microUS_3D, './microUS_3D_UF010.nii.gz')